In [12]:
import pandas as pd
import numpy as np
import os
import dpkt

In [14]:
test = pd.read_csv(r'D:\Test\trips\timeref.csv')
test = test.iloc[:5]

In [15]:
filelist = os.listdir(r'D:\Test')
filelist_ = []
for f in filelist:
    if len(f.split('.')) > 1:
        if f.split('.')[1] == 'pcap':
            filelist_.append(f)

In [16]:
date = [f.split('.')[0] for f in filelist_]
date_ = []
for d in date:
    if d[-1] == 'R':
        date_.append(d[:-2])
    else:
        date_.append(d)
date_ = pd.to_datetime(pd.Series(date_),format=('%Y-%m-%d-%H-%M-%S'))

In [17]:
Ts_records = pd.to_datetime(test.Timestamp,format=('%Y-%m-%d-%H-%M-%S'))

In [22]:
Pcap_inds = []
for i in range(len(Ts_records)):
    
    TimeDiff = (Ts_records.iloc[i] - date_)
    within30 = (TimeDiff <= pd.Timedelta(30,unit='Minute')) & ((TimeDiff >= pd.Timedelta(0,unit='Minute')))
    if within30.sum() == 0:
        Pcap_ind = -1
    else:
        Pcap_ind = TimeDiff.loc[within30].argmin()
    Pcap_inds.append(Pcap_ind)
Pcap_inds = np.array(Pcap_inds)
uni_ind = np.unique(Pcap_inds)
target_frames = []
for i in uni_ind:
    if i == -1:
        continue
    start_frames = np.array(test.loc[Pcap_inds==i].FrameIndex - 30).reshape(-1,1)
    end_frames = np.array(test.loc[Pcap_inds==i].FrameIndex + 30).reshape(-1,1)
    start_frames[start_frames < 0] = 0
    end_frames[end_frames > 17999] = 17999
    target_frames.append(np.concatenate([start_frames,end_frames],axis = 1))

In [28]:
Ts_records

0   2022-09-23 17:30:00
1   2022-09-23 17:30:00
2   2022-09-24 01:30:01
3   2022-09-24 01:30:02
4   2022-09-24 02:30:03
Name: Timestamp, dtype: datetime64[ns]

In [31]:
date_

0   2022-09-23 17:30:00
1   2022-09-23 18:59:05
2   2022-09-24 01:30:00
dtype: datetime64[ns]

In [43]:
TimeDiff = (Ts_records.iloc[2] - date_)
within30 = (TimeDiff <= pd.Timedelta(30,unit='Minute')) & ((TimeDiff >= pd.Timedelta(0,unit='Minute')))

In [53]:
TimeDiff.loc[within30].argsort().index[0]

2

In [39]:
within30

0    False
1    False
2    False
dtype: bool

In [23]:
Pcap_inds

array([ 0,  0,  0,  0, -1])